In [ ]:
# %load_ext autoreload
# %autoreload 2

In [10]:
import plb
from plb.envs import make
import taichi as ti

In [ ]:
from plt.envs.env import 

In [ ]:
# env = make("Move-v1")


env.reset()
taichi_env = env.taichi_env

In [ ]:
import taichi as ti
import numpy as np
from plb.engine.taichi_env import TaichiEnv
env.reset()
state = taichi_env.get_state()
sim_state = state['state']
actions = np.random.random(size=(50, 6))*0.01

def forward(taichi_env: TaichiEnv, sim_state, action):
    taichi_env.set_state(sim_state, 666, False)
    with ti.Tape(loss=taichi_env.loss.loss):
        for i in range(len(action)):
            taichi_env.step(action[i])
            loss_info = taichi_env.compute_loss()
    loss = taichi_env.loss.loss[None]
    return loss, taichi_env.primitives.get_grad(len(action))

loss, grad = forward(taichi_env, sim_state, actions)

print(loss)

In [ ]:
grad.shape